In [1]:
import numpy as np
import os
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import sweetviz as sv
import statsmodels.api as sm
from scipy import stats
from sklearn.model_selection import train_test_split
from math import sqrt
from sklearn.ensemble import RandomForestRegressor
from tabulate import tabulate
import warnings                                       # Remove unnecessary warnings for better interpretability 
warnings.filterwarnings('ignore')
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier

traindata_df = pd.read_excel('train.xlsx')
testdata_df = pd.read_excel('test.xlsx')

xdf = pd.DataFrame()
def propensity(traindata_df):
    
    traindata_df = traindata_df.drop(['previous','profit','id'], axis = 1)
    traindata_df = traindata_df.drop(index = traindata_df.index[-2:])
    
    traindata_df2 = traindata_df.copy()
    x = traindata_df['custAge'].median()
    traindata_df2['custAge'].fillna(x, inplace = True)
    traindata_df = traindata_df2
    
    traindata_df3 = traindata_df.copy()
    x = traindata_df['schooling'].mode()[0]
    traindata_df3['schooling'].fillna(x, inplace = True)
    traindata_df = traindata_df3
    
    traindata_df4 = traindata_df.copy()
    traindata_df4['day_of_week'].fillna("unknown", inplace = True)
    traindata_df = traindata_df4
   
    traindata_df = traindata_df.drop(['pmonths'], axis = 1)
    
    traindata_df['pdays'] = traindata_df['pdays'].apply(lambda x: -1 if x == 999 else x)
    
    traindata_df7 = traindata_df.copy()
    traindata_df = traindata_df7.drop(['campaign','housing','loan'], axis = 1)
    traindata_df = traindata_df.drop(['month','day_of_week'], axis = 1)
    
    schooling_enc = traindata_df['schooling'].unique()
    profession_enc = traindata_df['profession'].unique()
    marital_enc = traindata_df['marital'].unique()
    default_enc = traindata_df['default'].unique()
    contact_enc = traindata_df['contact'].unique()
    poutcome_enc = traindata_df['poutcome'].unique()
    categories = [profession_enc, marital_enc, schooling_enc, default_enc, contact_enc, poutcome_enc]
    
    onehot_encoder = OneHotEncoder(sparse=False, categories=categories)
    traindata_df_categorical = traindata_df[['profession','marital','schooling','default','contact','poutcome']]
    traindata_df_categorical = onehot_encoder.fit_transform(traindata_df_categorical)
    traindata_df_categorical = pd.DataFrame(traindata_df_categorical, columns=onehot_encoder.get_feature_names_out())
    traindata_df_categorical
    traindata_df=pd.concat([traindata_df,traindata_df_categorical],axis=1)
    traindata_df=traindata_df.drop(['profession','marital','schooling','default','contact','poutcome'],axis=1)
    traindata_df_dt = traindata_df[['custAge', 'pdays', 'emp.var.rate', 'cons.price.idx', 'cons.conf.idx',
       'euribor3m', 'nr.employed', 'pastEmail',
       'profession_admin.', 'profession_services', 'profession_blue-collar',
       'profession_entrepreneur', 'profession_technician',
       'profession_retired', 'profession_housemaid', 'profession_student',
       'profession_unknown', 'profession_unemployed',
       'profession_self-employed', 'profession_management', 'marital_single',
       'marital_divorced', 'marital_married', 'marital_unknown',
       'schooling_university.degree', 'schooling_high.school',
       'schooling_professional.course', 'schooling_basic.4y',
       'schooling_unknown', 'schooling_basic.9y', 'schooling_basic.6y',
       'schooling_illiterate', 'default_no', 'default_unknown', 'default_yes',
       'contact_cellular', 'contact_telephone', 'poutcome_nonexistent',
       'poutcome_failure', 'poutcome_success','responded']]
    xdt = traindata_df_dt.drop(['responded'], axis = 1)
    ydt = pd.DataFrame(traindata_df_dt['responded'])
    y = traindata_df['responded'].values
    x_train, x_test, y_train, y_test = train_test_split(xdt, ydt, test_size = 0.2, random_state = 27, shuffle = True, stratify = y)
    SC=StandardScaler()
    x_train.iloc[:,:8]= SC.fit_transform(x_train.iloc[:,:8])
    x_test.iloc[:,:8]= SC.fit_transform(x_test.iloc[:,:8])
    rf = RandomForestClassifier(n_estimators = 10,max_depth = 6,oob_score = True)
    rf.fit(x_train,y_train)
    rffit = rf.fit(x_train,y_train)
    rfpred = rf.predict_proba(testdata_df)[:,1]
    return rfpred
    
#   testdata_df=testdata_df.drop(['housing','loan','month', 'day_of_week','campaign','pmonths'],axis=1)

In [2]:

testdata_df = testdata_df.drop(['previous','id'], axis = 1)
testdata_df2 = testdata_df.copy()
x = testdata_df['custAge'].median()
testdata_df2['custAge'].fillna(x, inplace = True)
testdata_df = testdata_df2
testdata_df3 = testdata_df.copy()
x = testdata_df['schooling'].mode()[0]
testdata_df3['schooling'].fillna(x, inplace = True)
testdata_df = testdata_df3
testdata_df4 = testdata_df.copy()
testdata_df4['day_of_week'].fillna("unknown", inplace = True)
testdata_df = testdata_df4
testdata_df=testdata_df.drop(['housing','loan','month', 'day_of_week','campaign','pmonths'],axis=1)
schooling_enc1 = testdata_df['schooling'].unique()
profession_enc1 = testdata_df['profession'].unique()
marital_enc1 = testdata_df['marital'].unique()
default_enc1 = testdata_df['default'].unique()
contact_enc1 = testdata_df['contact'].unique()
poutcome_enc1 = testdata_df['poutcome'].unique()
categories1 = [profession_enc1, marital_enc1, schooling_enc1, default_enc1, contact_enc1, poutcome_enc1]

onehot_encoder = OneHotEncoder(sparse=False, categories=categories1)
testdata_df_categorical = testdata_df[['profession','marital','schooling','default','contact','poutcome']]
testdata_df_categorical = onehot_encoder.fit_transform(testdata_df_categorical)
testdata_df_categorical = pd.DataFrame(testdata_df_categorical, columns=onehot_encoder.get_feature_names_out())
testdata_df_categorical
testdata_df=pd.concat([testdata_df,testdata_df_categorical],axis=1)
testdata_df= testdata_df[['custAge', 'pdays', 'emp.var.rate', 'cons.price.idx', 'cons.conf.idx',
       'euribor3m', 'nr.employed', 'pastEmail', 'profession_admin.',
       'profession_services', 'profession_blue-collar',
       'profession_entrepreneur', 'profession_technician',
       'profession_retired', 'profession_housemaid', 'profession_student',
       'profession_unknown', 'profession_unemployed',
       'profession_self-employed', 'profession_management', 'marital_single',
       'marital_divorced', 'marital_married', 'marital_unknown',
       'schooling_university.degree', 'schooling_high.school',
       'schooling_professional.course', 'schooling_basic.4y',
       'schooling_unknown', 'schooling_basic.9y', 'schooling_basic.6y',
       'schooling_illiterate', 'default_no', 'default_unknown', 'default_yes',
       'contact_cellular', 'contact_telephone', 'poutcome_nonexistent',
       'poutcome_failure', 'poutcome_success']]
testdata_df['propensity'] = propensity(traindata_df)
testdata_df['propensity'] = np.where(testdata_df['propensity'] >= 0.5,1,0)
testdata_df

,custAge,pdays,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,pastEmail,profession_admin.,profession_services,...,schooling_illiterate,default_no,default_unknown,default_yes,contact_cellular,contact_telephone,poutcome_nonexistent,poutcome_failure,poutcome_success,propensity
0,38.0,999,-1.1,94.199,-37.5,0.886,4963.6,2,1.0,0.0,...,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0
1,35.0,3,-3.4,92.379,-29.8,0.788,5017.5,2,0.0,1.0,...,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0
2,50.0,999,-1.8,92.893,-46.2,1.327,5099.1,2,0.0,0.0,...,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0
3,30.0,999,1.4,93.444,-36.1,4.964,5228.1,0,1.0,0.0,...,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0
4,39.0,999,-0.1,93.200,-42.0,4.153,5195.8,0,0.0,1.0,...,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32945,38.0,999,1.1,93.994,-36.4,4.857,5191.0,0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0
32946,32.0,999,-1.8,92.893,-46.2,1.266,5099.1,0,0.0,0.0,...,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0
32947,32.0,999,-1.8,92.893,-46.2,1.299,5099.1,0,0.0,1.0,...,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0
32948,32.0,999,1.4,94.465,-41.8,4.959,5228.1,0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0


In [3]:
testdata_df.to_csv('funcprop.csv')